#Install libraries, define functions and import data to dataframe


In [3]:
# installation
! pip install requests
! pip install geopy
import io
import requests
from io import BytesIO
import pandas as pd
response = requests.get('https://docs.google.com/spreadsheets/d/1ZPL7sejPopQVya3lE6F4-E1nPxJgqXaZ80fM-Xh-cDI/export?format=csv&id=1ZPL7sejPopQVya3lE6F4-E1nPxJgqXaZ80fM-Xh-cDI&gid=0')
assert response.status_code == 200, 'Wrong status code'
data = response.content
# import data to dataframe
df = pd.read_csv(BytesIO(data), usecols=['Name','Type']) #unprocessed data

# print few rows
df.head()

,Name,Type
0,Banana,0.0
1,Bottle,2.0
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN


In [ ]:
# import
# essential libraries
import json
from urllib.request import urlopen

# storing and anaysis
import numpy as np
import pandas as pd

# visualization
import folium

# date and time
import time
from datetime import datetime

# requests for getting data file from Google Sheet link
import requests

# geopy + geocoder = Nominatim for searching location using OpenStreetMap data.
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Vietnam Cities Map", timeout=3)

In [ ]:
# get latest Vietnam SARS-CoV-2 | COVID-19 data
import io
from io import BytesIO

# get data from shared Google Sheet
response = requests.get('https://docs.google.com/spreadsheets/u/1/d/1vkvCEkZ8txrTmEldQGAycVVQbBHV-BwqTaCrxNYTtug/export?format=csv&id=1vkvCEkZ8txrTmEldQGAycVVQbBHV-BwqTaCrxNYTtug&gid=453484829')
assert response.status_code == 200, 'Wrong status code'
data = response.content

# import data to dataframe
df = pd.read_csv(BytesIO(data), usecols=['Case', 'Current Location', 'Confirmed', 'Recovered']) #unprocessed data

# print few rows
df.head()

,Case,Current Location,Confirmed,Recovered
0,BN_01,"Ho Chi Minh City,Vietnam",1,1
1,BN_02,"Ho Chi Minh City,Vietnam",1,1
2,BN_03,"Thanh Hoa,Vietnam",1,1
3,BN_04,"Vinh Phuc,Vietnam",1,1
4,BN_05,"Vinh Phuc,Vietnam",1,1


In [ ]:
# define function to get latitude, longitude
def latlongGet (addressStr):
    location = geolocator.geocode(addressStr)
    if location is None:
        print("Cannot find address", addressStr)
    else:
        return location.latitude, location.longitude;

#Nation Map

In [ ]:
nrow,_ = df.shape    
for i in range(nrow):
    addr = df.iloc[i,1]
    if (str(addr) == 'nan'):
        print('index = ', i, ' addr = ', addr, ' -> no address')
    else:
        s = addr.split(',')    #delimiter = ','
        state = len(s) - 2  #get province / state
        city = s.pop(state)
        df.at[i,'City'] = city
print(df.head())

    Case          Current Location  Confirmed  Recovered              City
0  BN_01  Ho Chi Minh City,Vietnam          1          1  Ho Chi Minh City
1  BN_02  Ho Chi Minh City,Vietnam          1          1  Ho Chi Minh City
2  BN_03         Thanh Hoa,Vietnam          1          1         Thanh Hoa
3  BN_04         Vinh Phuc,Vietnam          1          1         Vinh Phuc
4  BN_05         Vinh Phuc,Vietnam          1          1         Vinh Phuc


In [ ]:
# select only confirmed cases & not recovered yet
dfi = df[(df['Confirmed']==1) & (df['Recovered']==0)]
print(dfi)
print(dfi.shape)

       Case          Current Location  Confirmed  Recovered              City
90    BN_91  Ho Chi Minh City,Vietnam          1          0  Ho Chi Minh City
270  BN_271  Ho Chi Minh City,Vietnam          1          0  Ho Chi Minh City
288  BN_289         Thai Binh,Vietnam          1          0         Thai Binh
289  BN_290         Thai Binh,Vietnam          1          0         Thai Binh
291  BN_292         Thai Binh,Vietnam          1          0         Thai Binh
292  BN_293         Thai Binh,Vietnam          1          0         Thai Binh
295  BN_296         Thai Binh,Vietnam          1          0         Thai Binh
296  BN_297         Thai Binh,Vietnam          1          0         Thai Binh
297  BN_298         Thai Binh,Vietnam          1          0         Thai Binh
299  BN_300         Thai Binh,Vietnam          1          0         Thai Binh
300  BN_301         Thai Binh,Vietnam          1          0         Thai Binh
301  BN_302         Thai Binh,Vietnam          1          0     

In [ ]:
dfNation = pd.DataFrame(dfi,columns = ['Case', 'Current Location', 'Confirmed', 'Recovered','City'])
dfNation ['count'] = dfNation .groupby('City')['City'].transform('count')
dfNation.head()
# dfNation.info()

,Case,Current Location,Confirmed,Recovered,City,count
90,BN_91,"Ho Chi Minh City,Vietnam",1,0,Ho Chi Minh City,5
270,BN_271,"Ho Chi Minh City,Vietnam",1,0,Ho Chi Minh City,5
288,BN_289,"Thai Binh,Vietnam",1,0,Thai Binh,21
289,BN_290,"Thai Binh,Vietnam",1,0,Thai Binh,21
291,BN_292,"Thai Binh,Vietnam",1,0,Thai Binh,21


In [ ]:
# create map
# country center - position country map in the middle
centerLat = 16.4637 #Hue city Lat Long
centerLong = 107.5909

# display country map
m = folium.Map(location=[centerLat, centerLong], tiles='cartodbpositron',
               min_zoom=1, max_zoom=10, zoom_start=6)
#get no. of rows in dfi
nrow,_ = dfNation.shape
#display circles represent for patients
for i in range(0, nrow):
    time.sleep(1) #delay 1s to avoid #except OSError as err: # timeout error
    addr = dfNation .iloc[i,1]
    if (str(addr) == 'nan'):
        print('no address')
    else:
        lat, long = latlongGet(addr)
        #print(lat, long)
        folium.Circle(
        location=[lat, long],
        color='crimson', 
        tooltip =   '<li><bold>Province : '+str(dfNation .iloc[i]['City'])+
                    '<li><bold>Patients : '+str(dfNation .iloc[i]['count']),
        radius=int(dfNation.iloc[i]['count']**3)).add_to(m)  
    
#display map
m


#Ha Noi Map

In [ ]:
#HANOI MAP
dfi = dfi[dfi['City'] == 'Ha Noi']
print(dfi)

       Case         Current Location  Confirmed  Recovered    City
322  BN_323  Dong Anh,Ha Noi,Vietnam          1          0  Ha Noi


In [ ]:
dfHN=pd.DataFrame()
row,_ = dfi.shape
for j in range(row):
    add = dfi.iloc[j,1]
    add = str(add)
    print(add)
    d = add.split(',')
    districts= d.pop(0)
    #delimiter = ','
    dfHN.at[j,'Districts'] = districts
    dfHN.at[j,'Location'] = districts+', Ha Noi'
print(dfHN.head())

Dong Anh,Ha Noi,Vietnam
  Districts          Location
0  Dong Anh  Dong Anh, Ha Noi


In [ ]:
dfHN['count'] = dfHN.groupby('Districts')['Districts'].transform('count')
dfHN

,Districts,Location,count
0,Dong Anh,"Dong Anh, Ha Noi",1


In [ ]:
# create map
# country center - position country map in the middle
centerLat = 21.028511 #Hanoi Capital Lat Long
centerLong = 105.804817

# display country map
m = folium.Map(location=[centerLat, centerLong], tiles='cartodbpositron',
               min_zoom=1, max_zoom=15, zoom_start=10, )
#get no. of rows in dfHN
nrow,_ = dfHN.shape 
# print(nrow)
for i in range(0, nrow):
    time.sleep(1) #delay 1s to avoid #except OSError as err: # timeout error
    addr= dfHN.iloc[i,1]
    lat, long = latlongGet(addr)
    folium.Circle(
        location=[lat, long],
        color='crimson', 
        tooltip =   '<li><bold>District : '+str(dfHN .iloc[i]['Districts'])+
                    '<li><bold>Patients : '+str(dfHN .iloc[i]['count']),
        # radius=int(dfHN.iloc[i]['count']*10)).add_to(m)  
        radius=int(dfHN.iloc[i]['count']*1000)).add_to(m)
    
   
#display map
m